In [ ]:
# 1. IMPORT LIBRARY
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from statsmodels.graphics.tsaplots import plot_acf
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# 2. DEFINISIKAN PARAMETER
FILE_PATH = 'dataset_bbri.xlsx'
SHEET_NAME = 'Sheet1'
TARGET_COLUMN = 'Close' # Hanya fokus pada 'Close'

# Berapa hari ke belakang yang digunakan untuk memprediksi 1 hari ke depan
LOOK_BACK = 60 # Ini akan menjadi jumlah neuron di input layer

TRAIN_SPLIT_RATIO = 0.8
EPOCHS = 100
BATCH_SIZE = 32

In [ ]:
# 3. FUNGSI HELPER (Format data untuk MLP)
def create_dataset_mlp(dataset, look_back=1):
    """
    Mengubah array nilai menjadi format dataset untuk MLP.
    Contoh:
    dataset = [1, 2, 3, 4, 5]
    look_back = 2
    Maka hasilnya:
    dataX = [[1, 2], [2, 3], [3, 4]]
    dataY = [3, 4, 5]
    """
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        # Ambil sequence [i] s/d [i+look_back] sebagai fitur
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        # Ambil data [i + look_back] sebagai target
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
# 4. MUAT DAN PROSES DATA
print(f"Memuat data dari {FILE_PATH} (Fitur: {TARGET_COLUMN})...")
try:
    df = pd.read_excel(
        FILE_PATH,
        sheet_name=SHEET_NAME,
        usecols=['Date', TARGET_COLUMN], # Muat Tanggal + 1 fitur
        parse_dates=['Date'],
        index_col='Date',
        engine='openpyxl'
    )
except FileNotFoundError:
    print(f"Error: File '{FILE_PATH}' tidak ditemukan.")
    exit()
except ImportError:
    print("Error: Library 'openpyxl' tidak ditemukan.")
    exit()

df = df.dropna()
print("Data berhasil dimuat.")

# Ambil nilai 'Close' dan ubah menjadi numpy array
dataset_values = df[TARGET_COLUMN].values.reshape(-1, 1).astype('float32')

In [ ]:
# 5. NORMALISASI DATA
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_dataset = scaler.fit_transform(dataset_values)

In [ ]:
# 6. PISAHKAN DATA TRAINING DAN TESTING
train_size = int(len(scaled_dataset) * TRAIN_SPLIT_RATIO)
test_size = len(scaled_dataset) - train_size

train_data = scaled_dataset[0:train_size, :]
test_data = scaled_dataset[train_size:len(scaled_dataset), :]

print(f"Ukuran data training: {len(train_data)}")
print(f"Ukuran data testing: {len(test_data)}")

# Buat dataset X dan y
X_train, y_train = create_dataset_mlp(train_data, LOOK_BACK)
X_test, y_test = create_dataset_mlp(test_data, LOOK_BACK)

In [ ]:
# 7. BENTUK ULANG (RESHAPE) DATA
# Untuk MLP, kita tidak memerlukan reshape 3D.
# Input X sudah benar: (samples, features_in) -> (samples, LOOK_BACK)
print(f"Shape X_train (untuk MLP): {X_train.shape}")
print(f"Shape X_test (untuk MLP): {X_test.shape}")
print(f"Shape y_train: {y_train.shape}")

In [ ]:
# 8. BANGUN MODEL MLP (MULTILAYER PERCEPTRON)
print("Membangun model MLP...")

model = Sequential()
# Input layer (Dense) dengan input_dim = LOOK_BACK
model.add(Dense(units=100, input_dim=LOOK_BACK, activation='relu'))
# Hidden layer
model.add(Dense(units=50, activation='relu'))
# Output layer (1 neuron, tanpa aktivasi / linear)
model.add(Dense(units=1))

# Kompilasi model. 'adam' adalah optimizer yang menerapkan backpropagation
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

In [ ]:
# 9. LATIH MODEL
print(f"Mulai training model MLP (Epochs={EPOCHS}, Batch Size={BATCH_SIZE})...")
history = model.fit(
    X_train,
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_test, y_test),
    verbose=1
)
print("Training selesai.")

In [ ]:
# 10. LAKUKAN PREDIKSI
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

In [ ]:
# 11. KEMBALIKAN DATA KE SKALA SEMULA (INVERSE TRANSFORM)
# (Kembali ke cara inverse univariate yang sederhana)
train_predict = scaler.inverse_transform(train_predict)
y_train_orig = scaler.inverse_transform(y_train.reshape(-1, 1))

test_predict = scaler.inverse_transform(test_predict)
y_test_orig = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
# 12. HITUNG ROOT MEAN SQUARED ERROR (RMSE)
train_rmse = math.sqrt(mean_squared_error(y_train_orig, train_predict))
test_rmse = math.sqrt(mean_squared_error(y_test_orig, test_predict))

print(f"\n--- HASIL EVALUASI (MODEL MLP) ---")
print(f"Train RMSE: {train_rmse:.2f}")
print(f"Test RMSE:  {test_rmse:.2f}")

In [ ]:
# Hitung MAE
train_mae = mean_absolute_error(y_train_orig, train_predict)
test_mae = mean_absolute_error(y_test_orig, test_predict)

# Hitung MAPE (gunakan numpy.mean untuk menghindari error pembagian nol)
# Tambahkan epsilon kecil untuk menghindari pembagian dengan nol
epsilon = 1e-10
train_mape = np.mean(np.abs((y_train_orig - train_predict) / (y_train_orig + epsilon))) * 100
test_mape = np.mean(np.abs((y_test_orig - test_predict) / (y_test_orig + epsilon))) * 100


print(f"\n--- HASIL EVALUASI ---")
print(f"Train MAE: {train_mae:.2f}")
print(f"Test MAE:  {test_mae:.2f}")
print(f"Train MAPE: {train_mape:.2f}%")
print(f"Test MAPE:  {test_mape:.2f}%")

In [ ]:
# 13. VISUALISASIKAN HASIL PREDIKSI
print("Membuat plot hasil prediksi...")

# Siapkan data untuk plot
# Buat array kosong seukuran data asli, isi dengan NaN
train_predict_plot = np.empty_like(scaled_dataset)
train_predict_plot[:, :] = np.nan
# Isi bagian data training dengan hasil prediksi
train_predict_plot[LOOK_BACK:len(train_predict) + LOOK_BACK, :] = train_predict

# Lakukan hal yang sama untuk data testing
test_predict_plot = np.empty_like(scaled_dataset)
test_predict_plot[:, :] = np.nan
test_plot_start_index = train_size + LOOK_BACK
test_predict_plot[test_plot_start_index : test_plot_start_index + len(test_predict), :] = test_predict

# Plot
plt.figure(figsize=(16, 8))
# Plot data asli
plt.plot(df.index, scaler.inverse_transform(scaled_dataset), label='Data Asli (Close)')
# Plot prediksi training
plt.plot(df.index, train_predict_plot, label='Prediksi Training')
# Plot prediksi testing
plt.plot(df.index, test_predict_plot, label='Prediksi Testing')

plt.title(f'Prediksi Harga Saham (MLP) - Test RMSE: {test_rmse:.2f}')
plt.xlabel('Tanggal')
plt.ylabel('Harga Close')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 14. VISUALISASIKAN LOSS TRAINING VS VALIDASI
print("Membuat plot loss model...")
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Training & Validation Loss (Model MLP)')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
train_residuals = y_train_orig.flatten() - train_predict.flatten()

In [ ]:
print("Membuat plot autokorelasi residu training...")
plt.figure(figsize=(10, 6))
plot_acf(train_residuals, alpha=0.05)
plt.title('Autokorelasi Residu (ACF) - Model MLP')
plt.xlabel('Lag')
plt.ylabel('Autokorelasi')
plt.grid(True)
plt.show()

In [ ]:
test_residuals = y_test_orig.flatten() - test_predict.flatten()

In [ ]:
print("Membuat plot autokorelasi residu testing...")
plt.figure(figsize=(10, 6))
plot_acf(test_residuals, alpha=0.05)
plt.title('Autokorelasi Residu (ACF) - Model MLP')
plt.xlabel('Lag')
plt.ylabel('Autokorelasi')
plt.grid(True)
plt.show()